<a href="https://colab.research.google.com/github/opencoca/gimp-stable-diffusion/blob/main/gimp-stable-diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GIMP Stable Diffusion v1.0.3

###**IMPORTANT: Please use v1.0.3 of the GIMP plugin (current version in the github repository)** 
###The plugin version can be found at the top of the plugin file "gimp-stable-diffusion.py" which is located in your GIMP plugins folder.

Stable Diffusion notebook, which can be used together with the GIMP plugin to generate images.

Images are created with CompVis/Stability [Stable Diffusion](https://github.com/CompVis/stable-diffusion) with bonus [KLMS sampling](https://github.com/crowsonkb/k-diffusion.git) from [@RiversHaveWings](https://twitter.com/RiversHaveWings)

You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/CompVis/stable-diffusion).

By [@pharmapsychotic](https://twitter.com/pharmapsychotic). Modified by [@BlueTurtleAI](https://twitter.com/BlueTurtleAI) for the usage with the GIMP plugin.

In [ ]:
#@title Check GPU
!nvidia-smi -L

In [ ]:
#@title Mount Google Drive and Prepare Folders
from google.colab import drive
drive.mount('/content/gdrive')
outputs_path = "/content/gdrive/MyDrive/AI/StableDiffusion"
!mkdir -p $outputs_path
print(f"Outputs will be saved to {outputs_path}")

In [ ]:
#@title Installation
!pip install pytorch-lightning torch-fidelity
!pip install numpy omegaconf einops kornia pytorch-lightning
!pip install albumentations transformers
!pip install ftfy jsonmerge resize-right torchdiffeq tqdm
#!pip install pyngrok
#!pip install flask-ngrok

!pip install flask-cloudflared

!git clone https://github.com/CompVis/stable-diffusion
!git clone https://github.com/CompVis/taming-transformers
!git clone https://github.com/openai/CLIP.git
!git clone https://github.com/crowsonkb/k-diffusion.git

!echo '' > ./k-diffusion/k_diffusion/__init__.py


In [ ]:
#@title Load Model

#@markdown You need to get the model weights yourself and put on Google Drive or this Colab instance
checkpoint_model_file = "/content/gdrive/MyDrive/AI/models/sd-v1-4.ckpt" #@param {type:"string"}

checkpoint_model_url = "https://ipfs.io/ipfs/bafybeicrdgunwfjxm5yr7qqe5kgybaog65wnonymaeumzkto4eagrvwz2a/stable-diffusion-v1.4-and-license.zip"

import argparse, gc, json, os, random, sys, time, glob, requests
import torch
import torch.nn as nn
import numpy as np
import PIL
from contextlib import contextmanager, nullcontext
from einops import rearrange, repeat
from IPython.display import display, clear_output
from itertools import islice
from omegaconf import OmegaConf
from PIL import Image
from pytorch_lightning import seed_everything
from torch import autocast

sys.path.append("./CLIP")
sys.path.append('./k-diffusion')
sys.path.append('./stable-diffusion')
sys.path.append('./taming-transformers')

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

from k_diffusion.sampling import sample_lms
from k_diffusion.external import CompVisDenoiser

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

class config():
    def __init__(self):
        self.ckpt = checkpoint_model_file
        self.config = 'stable-diffusion/configs/stable-diffusion/v1-inference.yaml'
        self.ddim_eta = 0.0
        self.ddim_steps = 100
        self.fixed_code = True
        self.init_img = None
        self.n_samples = 1
        self.outdir = ""
        self.precision = 'autocast'
        self.prompt = ""
        self.sampler = 'klms'
        self.scale = 7.5
        self.seed = 42
        self.strength = 0.75 # strength for noising/unnoising. 1.0 corresponds to full destruction of information in init image
        self.H = 512
        self.W = 512
        self.C = 4
        self.f = 8

class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale

def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model = model.half().to(device)
    model.eval()
    return model
      
def load_img(path, shape):
    if path.startswith('http://') or path.startswith('https://'):
        image = Image.open(requests.get(path, stream=True).raw).convert('RGB')
    else:
        if os.path.isdir(path):
            files = [file for file in os.listdir(path) if file.endswith('.png') or file .endswith('.jpg')]
            path = os.path.join(path, random.choice(files))
            print(f"Chose random init image {path}")
        image = Image.open(path).convert('RGB')
    image = image.resize(shape, resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float16) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

opt = config()
config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")
model = model.to(device)
batch_idx = 0
sample_idx = 0

def generate(opt):
    global sample_idx
    seed_everything(opt.seed)
    os.makedirs(opt.outdir, exist_ok=True)

    if opt.sampler == 'plms':
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)

    model_wrap = CompVisDenoiser(model)       
    batch_size = opt.n_samples
    prompt = opt.prompt
    assert prompt is not None
    data = [batch_size * [prompt]]
    init_latent = None

    if opt.init_img != None and opt.init_img != '':
        init_image = load_img(opt.init_img, shape=(opt.W, opt.H)).to(device)
        init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
        init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space

    sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)

    t_enc = int(opt.strength * opt.ddim_steps)

    start_code = None
    if opt.fixed_code and init_latent == None:
        start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=device)

    images = []
    precision_scope = autocast if opt.precision == "autocast" else nullcontext
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                for prompts in data:
                    uc = None
                    if opt.scale != 1.0:
                        uc = model.get_learned_conditioning(batch_size * [""])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    c = model.get_learned_conditioning(prompts)

                    if init_latent != None:
                        z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                        samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                                unconditional_conditioning=uc,)
                    else:

                        if opt.sampler == 'klms':
                            print("Using KLMS sampling")
                            shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                            sigmas = model_wrap.get_sigmas(opt.ddim_steps)
                            model_wrap_cfg = CFGDenoiser(model_wrap)
                            x = torch.randn([opt.n_samples, *shape], device=device) * sigmas[0]
                            extra_args = {'cond': c, 'uncond': uc, 'cond_scale': opt.scale}
                            samples = sample_lms(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=False)
                        else:
                            shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                            samples, _ = sampler.sample(S=opt.ddim_steps,
                                                            conditioning=c,
                                                            batch_size=opt.n_samples,
                                                            shape=shape,
                                                            verbose=False,
                                                            unconditional_guidance_scale=opt.scale,
                                                            unconditional_conditioning=uc,
                                                            eta=opt.ddim_eta,
                                                            x_T=start_code)

                    x_samples = model.decode_first_stage(samples)
                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                        images.append(Image.fromarray(x_sample.astype(np.uint8)))
                        #filepath = os.path.join(opt.outdir, f"{batch_name}({batch_idx})_{sample_idx:04}.png")
                        #print(f"Saving to {filepath}")
                        #Image.fromarray(x_sample.astype(np.uint8)).save(filepath)
                        sample_idx += 1
    return images


In [ ]:
#@title Waiting for GIMP requests

from io import BytesIO
import json
import base64
from flask import Flask, Response, request, abort, make_response
#from flask_ngrok import run_with_ngrok
from flask_cloudflared import run_with_cloudflared

opt.sampler = "ddim"
opt.ddim_eta = 0.75
opt.n_samples = 1
opt.outdir = outputs_path

init_img = os.path.join(outputs_path, "init.png")
opt.init_img = init_img

os.makedirs(opt.outdir, exist_ok=True)
API_VERSION = 3

app = Flask(__name__)

@app.route("/api/img2img", methods=["POST"])
def img2img():
    r = request
    data = r.data.decode("utf-8")
    data = json.loads(data)

    api_version = 0

    if "api_version" in data:
       api_version = int(data["api_version"])

    if api_version != API_VERSION:
       abort(405)

    print("\n")
    print("Data sent from Gimp")
    print("init_strength: " + str(data["init_strength"]) + ", prompt_strength: " + str(data["prompt_strength"]) + ", steps: " + str(data["steps"]) + ", width: " + str(data["width"]) + ", height: " + str(data["height"]) + ", prompt: " + data["prompt"] + ", seed: " + str(data["seed"]) + ", api_version: " + str(data["api_version"]))
    print("\n")

    img_data = base64.b64decode(data["init_img"])
    img_file = open(init_img, "wb+")
    img_file.write(img_data)
    img_file.close()
    
    opt.W, opt.H = map(lambda x: x - x % 64, (int(data["width"]), int(data["height"])))
    opt.strength = max(0.0, min(1.0, 1.0 - float(data["init_strength"])))
    opt.guidance_scale = float(data["prompt_strength"])
    opt.ddim_steps = int(data["steps"])
    opt.prompt = data["prompt"]

    print("Parameters used for generating")
    print(opt.__dict__)
    print("\n")

    imgs_return = []

    for counter in range(data["image_count"]):
       # initialize
       gc.collect()
       torch.cuda.empty_cache()
       sample_idx = 0
    
       opt.seed = int(data["seed"]) if int(data["seed"]) != -1 else random.randint(0, 2**32)

       img = generate(opt)[0]

       img_data = BytesIO()
       img.save(img_data, format="PNG")
       img_data.seek(0)
       img_encoded = base64.b64encode(img_data.read())
       img_encoded = img_encoded.decode("utf-8")

       img_return = {"seed": opt.seed, "image": img_encoded}
       imgs_return.append(img_return)

       print(f"Used seed: {opt.seed}")

    data_return = {"images": imgs_return}
    data_return = json.dumps(data_return)

    if os.path.exists(init_img):
       os.remove(init_img)

    response = make_response()
    response.headers["mimetype"] = "application/json"
    response.data = data_return
    return response

run_with_cloudflared(app)
app.run()